In [1]:
# Cell 2 - create age and target, filter
CURRENT_YEAR = 2025
# target column (as found)
target_col = 'Q274: How many children do you have'
birth_year_col = 'Q261: Year of birth'  # exists in dataset preview
country_col = 'Q266: Country of birth: Respondent'  # numeric ISO code

# convert to numeric and handle bad/missing
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
df[birth_year_col] = pd.to_numeric(df[birth_year_col], errors='coerce')
df[country_col] = pd.to_numeric(df[country_col], errors='coerce')

# compute age
df['age'] = CURRENT_YEAR - df[birth_year_col]

# childbearing filter: 20-49 (common bracket) — adjust if needed
df_childbearing = df[(df['age'] >= 20) & (df['age'] <= 49)].copy()
print("childbearing rows:", df_childbearing.shape[0])

# Singapore subset (ISO numeric 702)
df_sg = df_childbearing[df_childbearing[country_col] == 702].copy()
print("Singapore rows (20-49):", df_sg.shape[0])

# global (all countries in same age bracket)
df_global = df_childbearing.copy()


NameError: name 'pd' is not defined

In [ ]:
# Cell 3 - EDA: target distribution and missingness
import matplotlib.pyplot as plt

# target distribution
plt.figure(figsize=(6,4))
df_childbearing[target_col].dropna().hist(bins=range(0,11))
plt.title('Distribution: number of children')
plt.xlabel('Number of children')
plt.ylabel('Count')
plt.show()

# missingness overview for top columns
miss = df_childbearing.isnull().mean().sort_values()
miss.head(30)


In [ ]:
# Example: auto-select some candidate features by keyword rules (adjust manually for best features)
value_keywords = ['Important', 'Duty', 'Worries', 'Satisfaction', 'Problem', 'attitude', 'value', 'worries', 'scale']
candidate_cols = [c for c in df.columns if any(k.lower() in c.lower() for k in ['important', 'duty', 'worries', 'satisfaction', 'problem', 'scale', 'chief wage', 'income', 'spend', 'household'])]
# ensure target + controls present
controls = ['age', 'Q270: Number of people in household']
candidate_cols = list(dict.fromkeys(candidate_cols + controls))
len(candidate_cols), candidate_cols[:40]


In [ ]:
# Cell 5A - Poisson using statsmodels (requires adding constant)
# Ensure required variables exist (features, X_sg_proc, y_sg). Fall back to sensible defaults.

# use candidate_cols (from earlier cell) if 'features' is not defined
if 'features' not in globals():
	features = candidate_cols  # candidate_cols defined in Cell 3

# Prepare X and y from the Singapore subset df_sg
X_sg = df_sg[features].copy()
y_sg = df_sg[target_col].copy()

# keep only rows with observed target
mask = y_sg.notna()
X_sg = X_sg.loc[mask]
y_sg = y_sg.loc[mask]

# simple preprocessing: impute numeric missings with column medians
X_sg_proc = X_sg.fillna(X_sg.median())

# add constant and fit Poisson GLM
X_sm = sm.add_constant(X_sg_proc)
poisson_model = sm.GLM(y_sg.values, X_sm, family=sm.families.Poisson()).fit()
print(poisson_model.summary())


In [ ]:
nb_model = sm.GLM(y_sg.values, X_sm, family=sm.families.NegativeBinomial()).fit()
print(nb_model.summary())


In [ ]:
# Cell 5B - RF regressor with cross-validation
rf = RandomForestRegressor(n_estimators=200, random_state=42)
scores = cross_val_score(rf, X_sg_proc, y_sg, cv=5, scoring='neg_mean_squared_error')
print("CV RMSE:", np.sqrt(-scores).mean())
# train final
rf.fit(X_sg_proc, y_sg)


In [ ]:
# Cell 5C - classification bucket
# Use the already-prepared y_sg (filtered) and X_sg_proc (processed) to build a classification target
y_class = pd.cut(y_sg, bins=[-1,0,1,100], labels=['0','1','2+'])

# RandomForestClassifier is already imported in an earlier cell; do not re-import
clf = RandomForestClassifier(n_estimators=200, random_state=42)
X_proc = X_sg_proc
clf.fit(X_proc, y_class)

print("Train acc:", clf.score(X_proc, y_class))


In [ ]:
# Simplified ranked SHAP plot: mean absolute SHAP per feature
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap

# Require: shap_values (from explainer) and features list
if 'shap_values' not in globals():
    raise RuntimeError('`shap_values` not found. Run the SHAP explainer cell first.')
if 'features' not in globals():
    raise RuntimeError('`features` not defined — run feature selection cells first.')

# Extract numeric SHAP matrix (handle common shap objects)
vals = shap_values.values if hasattr(shap_values, 'values') else shap_values
if isinstance(vals, list):
    vals = np.array([np.array(v) for v in vals])
    if vals.ndim == 3:
        vals = vals.mean(axis=0)
vals = np.asarray(vals)
if vals.ndim == 1:
    vals = vals.reshape(1, -1)

# Build DataFrame with feature names (best-effort)
try:
    df_shap = pd.DataFrame(vals, columns=features)
except Exception:
    nfeats = vals.shape[1]
    cols = features[:nfeats] if len(features) >= nfeats else [f'f{i}' for i in range(nfeats)]
    df_shap = pd.DataFrame(vals, columns=cols)
    print('Warning: features length mismatch — using best-effort column names')

# Compute mean absolute SHAP and plot ranked horizontal bar chart
mean_abs = df_shap.abs().mean().sort_values(ascending=True)
plt.figure(figsize=(8, max(4, 0.25 * len(mean_abs))))
mean_abs.plot(kind='barh', color='C0')
plt.xlabel('mean(|SHAP value|)')
plt.title('Feature importance — ranked by mean(|SHAP|)')
plt.tight_layout()
plt.show()

# Print top 10 features in descending order for quick reference
print('\nTop 10 features by mean(|SHAP|):')
print(mean_abs.sort_values(ascending=False).head(10).to_string())


In [ ]:
# Cell 7 - replicate modelling on global sample; compare feature importance
# Prepare global dataset same way:
global_df = df_global.dropna(subset=[target_col]).copy()
global_df = global_df[(global_df['age'] >= 20) & (global_df['age'] <= 49)]

# You may need to subset to rows with valid features
X_global = global_df[features].copy()
y_global = global_df[target_col]
X_global_proc = preprocessor.transform(X_global.fillna(X_global.median()))

# Train RF on global
rf_global = RandomForestRegressor(n_estimators=200, random_state=42)
rf_global.fit(X_global_proc, y_global)

# Compute permutation importances or SHAP for both and compare top 10
importances_sg = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False).head(15)
importances_global = pd.Series(rf_global.feature_importances_, index=features).sort_values(ascending=False).head(15)
print("SG top features:\n", importances_sg)
print("Global top features:\n", importances_global)


In [ ]:
# Cell 8 - interaction example
global_df['income_scale'] = global_df['Q288: Scale of incomes']  # rename as needed
# create an interaction term example: income_scale * 'Problem if women have more income than husband'
global_df['interaction_example'] = (global_df['Q288: Scale of incomes'].fillna(0) *
                                   global_df['Q35: Problem if women have more income than husband'].fillna(0))
# include interaction in model


In [ ]:
# Cell 9 - streamlit prototype sketch (save as app.py)
import streamlit as st
import pickle
# load your trained model & preprocessor
# model = pickle.load(open('rf_sg_model.pkl','rb'))
# preprocessor = pickle.load(open('preprocessor.pkl','rb'))

st.title("Family Planning Insight Tool (Prototype)")
age = st.number_input("Age", min_value=18, max_value=60, value=30)
income_scale = st.selectbox("Income scale", [1,2,3,4,5])
q_duty = st.slider("Duty towards society to have children (low-high)", 0, 10, 5)

if st.button("Get insight"):
    x = pd.DataFrame([{ 'age': age, 'Q288: Scale of incomes': income_scale, 'Q37: Duty towards society to have children': q_duty }])
    Xp = preprocessor.transform(x[features])
    pred_children = model.predict(Xp)[0]
    st.write(f"Modelled expected number of children: {pred_children:.2f}")
    # show SHAP explanation
    shap_vals = explainer(Xp)
    st.pyplot(shap.plots.force(shap_vals))
